# ref https://docs.sqlalchemy.org/en/14/orm/basic_relationships.html

# Database schema initalization

In [1]:
from sqlalchemy import orm, Column, Integer, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
import sqlalchemy as sa
from sqlalchemy.orm import relationship, sessionmaker


base = declarative_base()
engine = sa.create_engine('postgresql+psycopg2://postgres:password@db/postgres')
base.metadata.bind = engine
session = orm.scoped_session(orm.sessionmaker())(bind=engine)

class Recording(base):
    __tablename__ = 'recording' 
    id = sa.Column(sa.Integer,primary_key=True)
    location = sa.Column(sa.String(255))
    sensors = relationship("Sensordata", back_populates="recording")
    footages = relationship("Footage", back_populates="recording")
    

class Footage(base):
    __tablename__ = 'footage' 
    id = sa.Column(sa.Integer,primary_key=True)
    footage_type = sa.Column(sa.String(255))
    filename = sa.Column(sa.String(255))
    labels = relationship("Label", back_populates="footage")
    recording = relationship("Recording", back_populates="footages")
    recording_id = Column(Integer, ForeignKey('recording.id'))



class Sensordata(base):
    __tablename__ = 'sensor' 
    id = sa.Column(sa.Integer,primary_key=True)
    timestamp = sa.Column(sa.Integer)
    relative_timestamp = sa.Column(sa.Integer)
    metric_type = sa.Column(sa.String(255))
    metric_value = sa.Column(sa.Float)
    recording = relationship("Recording", back_populates="sensors")
    recording_id = Column(Integer, ForeignKey('recording.id'))

class Label(base):
    __tablename__ = 'label' 
    id = sa.Column(sa.Integer,primary_key=True)
    timestamp = sa.Column(sa.Integer)
    relative_timestamp = sa.Column(sa.Integer)
    footage = relationship("Footage", back_populates="labels")
    label_type = sa.Column(sa.String(255),)
    label_certainty = sa.Column(sa.Float)    
    footage_id = Column(Integer, ForeignKey('footage.id'))

base.metadata.create_all()

Session = sessionmaker(bind=engine) # session maker for connecting to the db later


# Full load database initalization

In [2]:
import random
import tqdm

In [3]:
NUMBER_OF_SECONDS_OF_DATA = 50
NUMBER_RECORDINGS = 3

LOCATION_A = 'Budapest'
LOCATION_B =  'Vienna'

SENSOR_TYPES = ['speed', 'acceleration']
LABEL_TYPES = ['pedestrian', 'animal', 'car', 'motor', 'street_sign']
FOOTAGE_TYPES = ['front', 'back']


RANDOM_UNIXTIMESTAMP_MIN = 1647270461 # Mon Mar 14 2022 15:07:41 GMT+0000
RANDOM_UNIXTIMESTAMP_MAX = 1647356861 # Tue Mar 15 2022 15:07:41 GMT+0000



session = Session()

with Session() as session:

    for _ in tqdm.tqdm(range(NUMBER_RECORDINGS)):
        # Create recording
        location = LOCATION_A if random.randint(0,1)  else LOCATION_B
        recording = Recording(location=location)
        start_time = random.randint(RANDOM_UNIXTIMESTAMP_MIN,RANDOM_UNIXTIMESTAMP_MAX)

        for x in range(NUMBER_OF_SECONDS_OF_DATA):
            # Create sensor data
            for sensor_type in SENSOR_TYPES:
                value = random.random() * 10 # float value beetween 0, 10
                sensor_data = Sensordata(timestamp=start_time+x, relative_timestamp=x, metric_type=sensor_type, metric_value=value)
                recording.sensors.append(sensor_data)

        # create Footage
        session.add(recording)
        session.commit()
        for footage_type in FOOTAGE_TYPES:
            filename = f'{recording.id}-{start_time}-{footage_type}.hdf5'
            footage = Footage(footage_type=footage_type, filename=filename, recording=recording)

            for x in range(NUMBER_OF_SECONDS_OF_DATA):

                for label_type in LABEL_TYPES:
                    if random.random() < 0.2: # not all frames or labeled

                        label = Label(timestamp=start_time+x, relative_timestamp=x, label_type=label_type, label_certainty=random.random())
                        footage.labels.append(label)



            session.add(footage)
            session.commit()


100%|██████████| 3/3 [00:01<00:00,  2.98it/s]


# query

## for each recfile list the 5 label classes with the highest confidence
### for footage


In [8]:
with Session() as session: # create a context, so the session will be cloed in the end
    query = """ 
    SELECT Footage.id, Footage.recording_id, Footage.footage_type, Footage.filename, label.label_type, label.max_label_certainty
    FROM footage
    INNER JOIN
        (SELECT label_type, footage_id, max(label_certainty) as max_label_certainty
        FROM Label
        GROUP BY label_type, footage_id
            ) label
    ON footage.id=label.footage_id
    """
    res = session.execute(query)


    for idx, row in enumerate(res):
        print(row)
        if idx >10: break # comment if all the values are needed



(20, 10, 'back', '10-1647296023-back.hdf', 'motor', 0.936175619818008)
(19, 10, 'front', '10-1647296023-front.hdf', 'motor', 0.8369814713970877)
(34, 17, 'back', '17-1647333371-back.hdf', 'motor', 0.9798287927143029)
(1, 1, 'front', '1-1647330719-front.hdf', 'car', 0.9634154455041045)
(36, 18, 'back', '18-1647275149-back.hdf', 'pedestrian', 0.924015692791155)
(40, 20, 'back', '20-1647307966-back.hdf', 'car', 0.9806641976041778)
(8, 4, 'back', '4-1647354068-back.hdf', 'animal', 0.9850040167382416)
(12, 6, 'back', '6-1647290149-back.hdf', 'motor', 0.9564814142968693)
(30, 15, 'back', '15-1647306949-back.hdf', 'animal', 0.9966556635422761)
(25, 13, 'front', '13-1647341541-front.hdf', 'street_sign', 0.9738128757319061)
(11, 6, 'front', '6-1647290149-front.hdf', 'car', 0.9867421686830888)
(24, 12, 'back', '12-1647271962-back.hdf', 'motor', 0.8346346950122534)


### for a recording in general

In [9]:
with Session() as session:
    query ="""
    SELECT *
    FROM recording
    INNER JOIN
        (SELECT Footage.id, Footage.recording_id, Footage.footage_type, Footage.filename, label.label_type, label.max_label_certainty
            FROM footage
            INNER JOIN
                (SELECT label_type, footage_id, max(label_certainty) as max_label_certainty
                FROM Label
                GROUP BY label_type, footage_id
                    ) label
            ON footage.id=label.footage_id
        ) inner_table
    ON recording.id = inner_table.recording_id
    """

    res = session.execute(query)


    for idx, row in enumerate(res):
        print(row)
        if idx >10: break # comment if all the values are needed


(1, 'Budapest', 1, 1, 'front', '1-1647330719-front.hdf', 'motor', 0.9438593555048512)
(1, 'Budapest', 2, 1, 'back', '1-1647330719-back.hdf', 'motor', 0.7797017971611884)
(1, 'Budapest', 2, 1, 'back', '1-1647330719-back.hdf', 'car', 0.958325173443521)
(1, 'Budapest', 1, 1, 'front', '1-1647330719-front.hdf', 'pedestrian', 0.9710128750528997)
(1, 'Budapest', 1, 1, 'front', '1-1647330719-front.hdf', 'street_sign', 0.9444887615143189)
(1, 'Budapest', 2, 1, 'back', '1-1647330719-back.hdf', 'animal', 0.9206772801231674)
(1, 'Budapest', 2, 1, 'back', '1-1647330719-back.hdf', 'pedestrian', 0.9278264015704876)
(1, 'Budapest', 1, 1, 'front', '1-1647330719-front.hdf', 'animal', 0.9955097698630524)
(1, 'Budapest', 2, 1, 'back', '1-1647330719-back.hdf', 'street_sign', 0.9347764433335419)
(1, 'Budapest', 1, 1, 'front', '1-1647330719-front.hdf', 'car', 0.9634154455041045)
(2, 'Budapest', 4, 2, 'back', '2-1647334174-back.hdf', 'motor', 0.9303953693659194)
(2, 'Budapest', 4, 2, 'back', '2-1647334174-bac

## find the recfile which has the most labels with at least 0.6 confidence

In [15]:
with Session() as session:

    query ='''
    SELECT Footage.id, Footage.footage_type, Footage.filename, l.label_type, label_certainty_count
    FROM footage
    INNER JOIN(
        SELECT label_type, count(label_certainty) as label_certainty_count, footage_id
            FROM Label
            WHERE label_certainty>0.6
            group by  label_type, footage_id
            ) l
    ON footage.id=l.footage_id
    ORDER BY label_certainty_count desc

    '''

    res = session.execute(query)


    for idx, row in enumerate(res):
        print(row)
        if idx >10: break # comment if all the values are needed
        

(36, 'back', '18-1647275149-back.hdf', 'street_sign', 16)
(35, 'front', '18-1647275149-front.hdf', 'street_sign', 15)
(28, 'back', '14-1647342929-back.hdf', 'car', 15)
(13, 'front', '7-1647322233-front.hdf', 'street_sign', 14)
(40, 'back', '20-1647307966-back.hdf', 'car', 14)
(33, 'front', '17-1647333371-front.hdf', 'car', 14)
(12, 'back', '6-1647290149-back.hdf', 'pedestrian', 13)
(36, 'back', '18-1647275149-back.hdf', 'animal', 13)
(40, 'back', '20-1647307966-back.hdf', 'animal', 13)
(23, 'front', '12-1647271962-front.hdf', 'car', 13)
(37, 'front', '19-1647327084-front.hdf', 'animal', 13)
(8, 'back', '4-1647354068-back.hdf', 'animal', 13)


In [14]:
with Session() as session:

    query ='''
    SELECT *
    FROM recording
    INNER JOIN(
            SELECT Footage.recording_id, Footage.id, Footage.footage_type, Footage.filename, l.label_type, label_certainty_count
            FROM footage
            INNER JOIN(
                SELECT label_type, count(label_certainty) as label_certainty_count, footage_id
                    FROM Label
                    WHERE label_certainty>0.6
                    group by  label_type, footage_id
                    ) l
            ON footage.id=l.footage_id
        ) inner_select
    ON inner_select.recording_id = recording.id
    ORDER BY label_certainty_count desc
    '''

    res = session.execute(query)


    for idx, row in enumerate(res):
        print(row)
        if idx >10: break # comment if all the values are needed

(18, 'Vienna', 18, 36, 'back', '18-1647275149-back.hdf', 'street_sign', 16)
(18, 'Vienna', 18, 35, 'front', '18-1647275149-front.hdf', 'street_sign', 15)
(14, 'Vienna', 14, 28, 'back', '14-1647342929-back.hdf', 'car', 15)
(20, 'Vienna', 20, 40, 'back', '20-1647307966-back.hdf', 'car', 14)
(7, 'Vienna', 7, 13, 'front', '7-1647322233-front.hdf', 'street_sign', 14)
(17, 'Budapest', 17, 33, 'front', '17-1647333371-front.hdf', 'car', 14)
(20, 'Vienna', 20, 40, 'back', '20-1647307966-back.hdf', 'animal', 13)
(18, 'Vienna', 18, 36, 'back', '18-1647275149-back.hdf', 'animal', 13)
(19, 'Budapest', 19, 37, 'front', '19-1647327084-front.hdf', 'animal', 13)
(6, 'Vienna', 6, 12, 'back', '6-1647290149-back.hdf', 'pedestrian', 13)
(4, 'Vienna', 4, 8, 'back', '4-1647354068-back.hdf', 'animal', 13)
(12, 'Budapest', 12, 23, 'front', '12-1647271962-front.hdf', 'car', 13)
